Comparison between counts in Halos for these three scenarios:
* No satellite cuts
* Not satellite at z=0
* Never satellite
* Not satellite at z<8

Also use vlsr<90 cut instead of vdev

In [1]:
import os
import sys
import time

import numpy as np

from scipy import stats

from astropy import units as u
from astropy import constants as cnst
from astropy.coordinates import SkyCoord, Distance
from astropy import modeling
from astropy import table
from astropy.table import Table
from astropy.io import fits

%matplotlib inline
from matplotlib import rcParams, style
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

rcParams['image.cmap'] = 'viridis'
rcParams['image.origin'] = 'lower'
rcParams['axes.prop_cycle'] = style.library['seaborn-deep']['axes.prop_cycle']
rcParams['figure.figsize'] = (14, 8)
rcParams['axes.titlesize'] =  rcParams['axes.labelsize'] = 16
rcParams['xtick.labelsize'] =  rcParams['ytick.labelsize'] = 14

In [2]:
import data_loaders
import detectability

In [3]:
from importlib import reload
first = lambda x: next(iter(x))

# Relevant Lookback times -> redshifts 

In [4]:
from astropy.cosmology import Planck15, z_at_value

In [5]:
Planck15.lookback_time(8)

<Quantity 13.158851036570463 Gyr>

In [6]:
Planck15.lookback_time(5)

<Quantity 12.624238411321304 Gyr>

In [7]:
z_at_value(Planck15.lookback_time, 3*u.Gyr)

0.24655286227260514

In [8]:
z_at_value(Planck15.lookback_time, 1*u.Gyr)

0.073015434533676843

# Load things

In [9]:
elvii_pairs = data_loaders.load_elvii_z0(isolated=False)

Loading /Users/erik/projects/m31-galfa/elvis_data/PairedCatalogs/Hall&Oates.txt
Loading /Users/erik/projects/m31-galfa/elvis_data/PairedCatalogs/Hamilton&Burr.txt
Loading /Users/erik/projects/m31-galfa/elvis_data/PairedCatalogs/Kek&Kauket.txt
Loading /Users/erik/projects/m31-galfa/elvis_data/PairedCatalogs/Lincoln&Douglas.txt
Loading /Users/erik/projects/m31-galfa/elvis_data/PairedCatalogs/Orion&Taurus.txt
Loading /Users/erik/projects/m31-galfa/elvis_data/PairedCatalogs/Romulus&Remus.txt
Loading /Users/erik/projects/m31-galfa/elvis_data/PairedCatalogs/Scylla&Charybdis.txt
Loading /Users/erik/projects/m31-galfa/elvis_data/PairedCatalogs/Serena&Venus.txt
Loading /Users/erik/projects/m31-galfa/elvis_data/PairedCatalogs/Siegfried&Roy.txt
Loading /Users/erik/projects/m31-galfa/elvis_data/PairedCatalogs/Sonny&Cher.txt
Loading /Users/erik/projects/m31-galfa/elvis_data/PairedCatalogs/Thelma&Louise.txt
Loading /Users/erik/projects/m31-galfa/elvis_data/PairedCatalogs/Zeus&Hera.txt


In [10]:
res = data_loaders.load_galfa_sensitivity('onempc_mass_sens.fits.zip')
galfa_sens, galfa_sens_scs, galfa_sens_wcs, galfa_sens_hdu = res

/Users/erik/miniconda3/envs/anaconda/lib/python3.5/site-packages/astropy/coordinates/angles.py:656: RuntimeWarning: invalid value encountered in less
  if np.any(self_angle < wrap_angle_floor) or np.any(self_angle >= wrap_angle):
/Users/erik/miniconda3/envs/anaconda/lib/python3.5/site-packages/astropy/coordinates/angles.py:656: RuntimeWarning: invalid value encountered in greater_equal
  if np.any(self_angle < wrap_angle_floor) or np.any(self_angle >= wrap_angle):
/Users/erik/miniconda3/envs/anaconda/lib/python3.5/site-packages/astropy/coordinates/angles.py:547: RuntimeWarning: invalid value encountered in less
  if np.any(angles.value < lower) or np.any(angles.value > upper):
/Users/erik/miniconda3/envs/anaconda/lib/python3.5/site-packages/astropy/coordinates/angles.py:547: RuntimeWarning: invalid value encountered in greater
  if np.any(angles.value < lower) or np.any(angles.value > upper):


In [11]:
elvii_trees = data_loaders.load_elvii_trees('ID,Mvir,Vmax,Rvir,upID,scale')

Loading /Users/erik/projects/m31-galfa/elvis_data/PairedTrees/Hall&Oates
Loading /Users/erik/projects/m31-galfa/elvis_data/PairedTrees/Hamilton&Burr
Loading /Users/erik/projects/m31-galfa/elvis_data/PairedTrees/Kek&Kauket
Loading /Users/erik/projects/m31-galfa/elvis_data/PairedTrees/Lincoln&Douglas
Loading /Users/erik/projects/m31-galfa/elvis_data/PairedTrees/Orion&Taurus
Loading /Users/erik/projects/m31-galfa/elvis_data/PairedTrees/Romulus&Remus
Loading /Users/erik/projects/m31-galfa/elvis_data/PairedTrees/Scylla&Charybdis
Loading /Users/erik/projects/m31-galfa/elvis_data/PairedTrees/Serena&Venus
Loading /Users/erik/projects/m31-galfa/elvis_data/PairedTrees/Siegfried&Roy
Loading /Users/erik/projects/m31-galfa/elvis_data/PairedTrees/Sonny&Cher
Loading /Users/erik/projects/m31-galfa/elvis_data/PairedTrees/Thelma&Louise
Loading /Users/erik/projects/m31-galfa/elvis_data/PairedTrees/Zeus&Hera


/Users/erik/projects/m31-galfa/data_loaders.py:83: RuntimeWarning: divide by zero encountered in true_divide
  coldct['z'] = 1./coldct['scale'] - 1


In [12]:
data_loaders.annotate_z0_from_trees(elvii_pairs, elvii_trees)
data_loaders.annotate_z0_from_trees(elvii_pairs, elvii_trees, zthresh=8)
data_loaders.annotate_z0_from_trees(elvii_pairs, elvii_trees, zthresh=5)
data_loaders.annotate_z0_from_trees(elvii_pairs, elvii_trees, zthresh=1)
data_loaders.annotate_z0_from_trees(elvii_pairs, elvii_trees, zthresh=0.5)
data_loaders.annotate_z0_from_trees(elvii_pairs, elvii_trees, zthresh=0.25)
data_loaders.annotate_z0_from_trees(elvii_pairs, elvii_trees, zthresh=0.1)

# Compute detectability 

In [13]:
#m31 = SkyCoord.from_name('M31')
m31 = SkyCoord(10.6847083*u.deg, 41.26875*u.deg, 700*u.kpc)

for tab in elvii_pairs.values():
    data_loaders.add_oriented_radecs(tab, 0, 1, target_coord=m31)
    data_loaders.add_oriented_radecs(tab, 1, 0, target_coord=m31)

/Users/erik/miniconda3/envs/anaconda/lib/python3.5/site-packages/astropy/units/quantity.py:782: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return super(Quantity, self).__eq__(other)


In [14]:
detectability.compute_elvis_mgas_bradford_15(elvii_pairs, 'Mstar_preferred')

In [15]:
detectability.compute_elvis_detectability(galfa_sens, galfa_sens_wcs, 'dr1', elvii_pairs)

In [16]:
wakker_hvcs = Table.read('wakk.fits')
wakker_scs = SkyCoord(l=wakker_hvcs['L'], b=wakker_hvcs['B'], unit=u.deg, frame='galactic')

detectability.compute_elvis_findable(elvii_pairs, wakker_scs, wakker_hvcs['VLSR']*u.km/u.s)

# Now build table with the various scenarios 

In [17]:
tab = Table()
tab['name'] = np.array([], dtype='S20')
tab['n_all'] = np.array([], dtype=int)
tab['n_notsat'] = np.array([], dtype=int)
tab['n_neversat'] = np.array([], dtype=int)
tab['n_neversatz8'] = np.array([], dtype=int)
tab['n_neversatz5'] = np.array([], dtype=int)
tab['n_neversatz1'] = np.array([], dtype=int)
tab['n_neversatz0p5'] = np.array([], dtype=int)
tab['n_neversatz0p1'] = np.array([], dtype=int)


for pairnm, pair in elvii_pairs.items():
    for i in (0, 1):
        nm = pairnm.split('&')[i]
        
        findable = pair['findable_dr1_host' + str(i)]
        nall = np.sum(findable)
        nnotsat = np.sum(findable & ~pair['sat_of_either'])
        nneversat = np.sum(findable & ~pair['upIDever_either'])
        nneversatz8 = np.sum(findable & ~pair['upIDz8_either'])
        nneversatz5 = np.sum(findable & ~pair['upIDz5_either'])
        nneversatz1 = np.sum(findable & ~pair['upIDz1_either'])
        nneversatz0p5 = np.sum(findable & ~pair['upIDz0p5_either'])
        nneversatz0p1 = np.sum(findable & ~pair['upIDz0p1_either'])
        
        tab.add_row([nm, nall, nnotsat, nneversat, nneversatz8, nneversatz5, nneversatz1, nneversatz0p5, nneversatz0p1])
        
tab.show_in_notebook()

idx,name,n_all,n_notsat,n_neversat,n_neversatz8,n_neversatz5,n_neversatz1,n_neversatz0p5,n_neversatz0p1
0,Sonny,36,11,5,5,5,5,5,8
1,Cher,63,11,4,4,4,4,4,9
2,Serena,61,19,12,12,12,12,12,16
3,Venus,105,43,19,19,19,19,19,36
4,Lincoln,62,26,9,9,9,9,12,22
5,Douglas,58,20,6,6,6,6,7,14
6,Siegfried,65,36,17,17,17,17,17,29
7,Roy,80,29,13,13,13,13,13,22
8,Zeus,54,12,6,6,6,6,6,12
9,Hera,80,21,7,7,7,7,10,18


# Now repeat instead using vlsr cut 

In [18]:
# this changes the "findable" table entries, so it will change things if you then go back and execute the above cell
detectability.compute_elvis_findable(elvii_pairs, wakker_scs, wakker_hvcs['VLSR']*u.km/u.s, vlsr_thresh=90*u.km/u.s)

In [19]:
tab = Table()
tab['name'] = np.array([], dtype='S20')
tab['n_all'] = np.array([], dtype=int)
tab['n_notsat'] = np.array([], dtype=int)
tab['n_neversat'] = np.array([], dtype=int)
tab['n_neversatz8'] = np.array([], dtype=int)
tab['n_neversatz5'] = np.array([], dtype=int)
tab['n_neversatz1'] = np.array([], dtype=int)
tab['n_neversatz0p5'] = np.array([], dtype=int)
tab['n_neversatz0p25'] = np.array([], dtype=int)
tab['n_neversatz0p1'] = np.array([], dtype=int)


for pairnm, pair in elvii_pairs.items():
    for i in (0, 1):
        nm = pairnm.split('&')[i]
        
        findable = pair['findable_dr1_host' + str(i)]
        nall = np.sum(findable)
        nnotsat = np.sum(findable & ~pair['sat_of_either'])
        nneversat = np.sum(findable & ~pair['upIDever_either'])
        nneversatz8 = np.sum(findable & ~pair['upIDz8_either'])
        nneversatz5 = np.sum(findable & ~pair['upIDz5_either'])
        nneversatz1 = np.sum(findable & ~pair['upIDz1_either'])
        nneversatz0p5 = np.sum(findable & ~pair['upIDz0p5_either'])
        nneversatz0p25 = np.sum(findable & ~pair['upIDz0p25_either'])
        nneversatz0p1 = np.sum(findable & ~pair['upIDz0p1_either'])
        
        tab.add_row([nm, nall, nnotsat, nneversat, nneversatz8, nneversatz5, nneversatz1, 
                     nneversatz0p5, nneversatz0p25, nneversatz0p1])
        
tab.show_in_notebook()

idx,name,n_all,n_notsat,n_neversat,n_neversatz8,n_neversatz5,n_neversatz1,n_neversatz0p5,n_neversatz0p25,n_neversatz0p1
0,Sonny,26,9,3,3,3,3,3,4,6
1,Cher,46,13,6,6,6,6,6,7,10
2,Serena,33,12,8,8,8,8,8,11,11
3,Venus,80,36,18,18,18,18,18,20,30
4,Lincoln,38,15,1,1,1,1,3,5,12
5,Douglas,45,16,5,5,5,5,7,8,10
6,Siegfried,45,25,9,9,9,9,9,9,20
7,Roy,61,29,10,10,10,10,10,14,23
8,Zeus,39,11,7,7,7,7,7,7,11
9,Hera,62,17,7,7,7,7,11,15,17
